In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.cm import rainbow
import seaborn as sns
from scipy import stats
from scipy.stats import skew
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


## EDA

In [2]:
df = pd.read_csv('application_data.csv')

In [3]:
# Etapa 01 - Tratamento e limpeza dos dados nulos

docs_colunas = ['FLAG_DOCUMENT_' + str(i) for i in range(2, 22)]
df['NUM_DOCS_APRESENTADOS'] = df[docs_colunas].sum(axis=1)
flag_docs_colunas = [col for col in df.columns if col.startswith('FLAG_DOCUMENT_')]
df_exclusao_cols = df.drop(columns=flag_docs_colunas)
df = df_exclusao_cols

# Exclusão de colunas com valores nulos

def excluir_colunas_nulas(df, percentual):
    dados_faltantes = df.isnull().mean() * 100
    col_para_apagar = dados_faltantes[dados_faltantes > percentual].index.tolist()
    return col_para_apagar

percentual = 0
col_para_apagar = excluir_colunas_nulas(df, percentual)

# Exclui as colunas identificadas
df_exclusao_colunas = df.drop(columns=col_para_apagar)
df = df_exclusao_colunas

# Exclusão de colunas de categoria única

columns_to_drop = ['FLAG_MOBIL',
                   'FLAG_EMP_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_EMAIL','REG_REGION_NOT_WORK_REGION',
                   'REG_REGION_NOT_LIVE_REGION','REG_CITY_NOT_WORK_CITY','REG_CITY_NOT_LIVE_CITY']
df = df.drop(columns_to_drop,axis=1)

# Exclusão dos valores 'XNA' (essencialmente nulos) em CODE_GENDER

df = df[df['CODE_GENDER'] != 'XNA']

# Conversao de algumas colunas para anos

# Colunas que terão o sinal removido (valores absolutos)
colunas_para_remover_sinal = ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH']
df[colunas_para_remover_sinal] = df[colunas_para_remover_sinal].abs()

# Engenharia de atributos
df['IDADE'] = df['DAYS_BIRTH'] // 365
df['ANOS_TRABALHADOS'] = df['DAYS_EMPLOYED'] // 365
df['ANOS_REGISTRO'] = df['DAYS_REGISTRATION'] // 365
df['ANOS_PUBLICACAO_ID'] = df['DAYS_ID_PUBLISH'] // 365

# Remover colunas originais
df = df.drop(colunas_para_remover_sinal, axis=1)

#  Devido a uma alta correlação, decidi remover Region_Rating_Client_With_City

column_to_drop = ['REGION_RATING_CLIENT_W_CITY']
df1 = df.drop(column_to_drop,axis=1)

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 307507 entries, 0 to 307510
Data columns (total 27 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   SK_ID_CURR                   307507 non-null  int64  
 1   TARGET                       307507 non-null  int64  
 2   NAME_CONTRACT_TYPE           307507 non-null  object 
 3   CODE_GENDER                  307507 non-null  object 
 4   FLAG_OWN_CAR                 307507 non-null  object 
 5   FLAG_OWN_REALTY              307507 non-null  object 
 6   CNT_CHILDREN                 307507 non-null  int64  
 7   AMT_INCOME_TOTAL             307507 non-null  float64
 8   AMT_CREDIT                   307507 non-null  float64
 9   NAME_INCOME_TYPE             307507 non-null  object 
 10  NAME_EDUCATION_TYPE          307507 non-null  object 
 11  NAME_FAMILY_STATUS           307507 non-null  object 
 12  NAME_HOUSING_TYPE            307507 non-null  object 
 13  REGI

## Árvore 1 - Base após EDA

In [5]:
# Selecionar colunas numéricas (int64 e float64)
colunas_numericas = df1.select_dtypes(include=['int64', 'float64']).columns

df_arvore_1 = df1[colunas_numericas]
print('base para árvore: ', df_arvore_1.info())

<class 'pandas.core.frame.DataFrame'>
Index: 307507 entries, 0 to 307510
Data columns (total 17 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   SK_ID_CURR                   307507 non-null  int64  
 1   TARGET                       307507 non-null  int64  
 2   CNT_CHILDREN                 307507 non-null  int64  
 3   AMT_INCOME_TOTAL             307507 non-null  float64
 4   AMT_CREDIT                   307507 non-null  float64
 5   REGION_POPULATION_RELATIVE   307507 non-null  float64
 6   FLAG_WORK_PHONE              307507 non-null  int64  
 7   FLAG_PHONE                   307507 non-null  int64  
 8   REGION_RATING_CLIENT         307507 non-null  int64  
 9   HOUR_APPR_PROCESS_START      307507 non-null  int64  
 10  LIVE_REGION_NOT_WORK_REGION  307507 non-null  int64  
 11  LIVE_CITY_NOT_WORK_CITY      307507 non-null  int64  
 12  NUM_DOCS_APRESENTADOS        307507 non-null  int64  
 13  IDAD

In [6]:
# Passo 1: Preparação dos Dados
# Passo 2: Treinamento da Árvore de Decisão
# Passo 3: Avaliação do Modelo

# Definir as features e o target
X = df_arvore_1.drop(columns=['TARGET'])  # Todas as variáveis menos o target
y = df_arvore_1['TARGET']  # O target

# Dividir os dados em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar o modelo de árvore de decisão
decision_tree = DecisionTreeClassifier(random_state=42)

# Treinar o modelo com os dados de treino
decision_tree.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred = decision_tree.predict(X_test)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Exibir os resultados
print(f'Acurácia: {accuracy:.4f}')
print(f'Precisão: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')
print('\nMatriz de Confusão:')
print(confusion_matrix(y_test, y_pred))
print('\nRelatório de Classificação:')
print(classification_report(y_test, y_pred))


Acurácia: 0.8403
Precisão: 0.1076
Recall: 0.1310
F1-Score: 0.1181

Matriz de Confusão:
[[51021  5460]
 [ 4363   658]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.92      0.90      0.91     56481
           1       0.11      0.13      0.12      5021

    accuracy                           0.84     61502
   macro avg       0.51      0.52      0.52     61502
weighted avg       0.85      0.84      0.85     61502



In [8]:
# Treinamento da árvore com class_weight balanceado
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Parâmetros a serem ajustados
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced']
}

# Modelo de árvore
decision_tree = DecisionTreeClassifier(random_state=42)

# Grid Search com validação cruzada
grid_search = GridSearchCV(estimator=decision_tree, param_grid=param_grid, cv=5, scoring='f1', verbose=1, n_jobs=-1)

# Treinamento
grid_search.fit(X_train, y_train)

# Avaliação
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print(f"Melhores parâmetros: {grid_search.best_params_}")
print(f'Acurácia: {accuracy_score(y_test, y_pred):.4f}')
print(f'Precisão: {precision_score(y_test, y_pred):.4f}')
print(f'Recall: {recall_score(y_test, y_pred):.4f}')
print(f'F1-Score: {f1_score(y_test, y_pred):.4f}')

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Melhores parâmetros: {'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}
Acurácia: 0.5877
Precisão: 0.1129
Recall: 0.5909
F1-Score: 0.1896


Ao analisar os resultados das duas abordagens para a detecção de fraudes:

### Primeira Árvore (sem `class_weight` balanceado):
- **Acurácia**: 0.8403
- **Precisão**: 0.1076
- **Recall**: 0.1310
- **F1-Score**: 0.1181

#### Interpretação:
- Alta acurácia (0.84), mas isso é esperado, já que a classe "não fraude" é majoritária, e o modelo está classificando corretamente a maioria das instâncias "não fraude".
- **Baixa precisão e recall para fraudes** (classe 1): O modelo tem dificuldade em identificar fraudes, resultando em muitos falsos negativos e falsos positivos.
- **F1-Score baixo** (0.1181): O modelo não está conseguindo equilibrar bem a precisão e o recall, o que o torna menos eficiente na detecção de fraudes.

### Segunda Árvore (com `class_weight` balanceado):
- **Acurácia**: 0.5877
- **Precisão**: 0.1129
- **Recall**: 0.5909
- **F1-Score**: 0.1896

#### Interpretação:
- A **acurácia diminuiu** significativamente (0.5877), mas isso era esperado, pois o foco agora é melhorar a detecção da classe minoritária (fraudes), e o modelo deixa de priorizar a classe "não fraude".
- **Recall** para fraudes **melhorou muito** (0.5909): O modelo agora está capturando muito mais fraudes, o que é importante, pois é preferível detectar mais casos de fraude, mesmo que isso implique em mais falsos positivos.
- **Precisão** continua baixa (0.1129), o que indica que, embora o modelo esteja capturando mais fraudes, ainda há muitos falsos positivos.
- **F1-Score aumentou** (0.1896), o que mostra uma melhoria no equilíbrio entre precisão e recall, mas ainda é um valor relativamente baixo para fraudes.

### Análise Geral:
- **Primeira Árvore**: Tem uma alta acurácia, mas um desempenho muito ruim na detecção de fraudes, com baixa precisão e recall.
- **Segunda Árvore (com `class_weight` balanceado)**: Focou melhor na detecção de fraudes, com um recall significativamente maior, o que é crucial para problemas de fraudes. No entanto, isso custou uma queda acentuada na acurácia geral e a precisão ainda é baixa.

### Conclusão:
A segunda abordagem, com `class_weight='balanced'`, apesar de ter uma acurácia geral mais baixa, é **mais adequada para a detecção de fraudes**, pois aumenta consideravelmente o recall, permitindo capturar mais casos de fraude. No entanto, há espaço para melhorias, principalmente em termos de precisão, para reduzir falsos positivos. Ajustes adicionais nos hiperparâmetros ou o uso de outras técnicas, como ensemble methods (e.g., Random Forest ou XGBoost), podem melhorar o equilíbrio entre precisão e recall.

## Árvore 2 - Base completa

In [9]:
# Carregar os dados
df2 = pd.read_csv('application_data.csv')

In [10]:
variables_to_test = df2.select_dtypes(include=['int64', 'float64']).columns

alpha = 0.05
significant_variables = []

for variable in variables_to_test:
    # Extrair dados para os grupos de inadimplência (TARGET = 1) e não inadimplência (TARGET = 0)
    data_default = df2[df2['TARGET'] == 1][variable].dropna()  # Remover valores nulos
    data_no_default = df2[df2['TARGET'] == 0][variable].dropna()  # Remover valores nulos

    # Realizar o teste t para duas amostras
    t_statistic, p_value = stats.ttest_ind(data_default, data_no_default, equal_var=False)

    # Verificar significância
    if p_value < alpha:
        significant_variables.append(variable)

# Criar um novo DataFrame apenas com as variáveis significativas
df20 = df2[significant_variables]

def identify_columns_to_drop(df, percentage):
    missing_percentage = df.isnull().mean() * 100
    columns_to_drop = missing_percentage[missing_percentage > percentage].index.tolist()
    return columns_to_drop

percentage = 0
columns_to_drop = identify_columns_to_drop(df20, percentage)

# Exclui as colunas identificadas
df_dropped = df20.drop(columns=columns_to_drop)
df_arvore_2 = df_dropped

#print('base árvore 2: ', df_arvore_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 86 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   TARGET                        307511 non-null  int64  
 1   CNT_CHILDREN                  307511 non-null  int64  
 2   AMT_CREDIT                    307511 non-null  float64
 3   AMT_ANNUITY                   307499 non-null  float64
 4   AMT_GOODS_PRICE               307233 non-null  float64
 5   REGION_POPULATION_RELATIVE    307511 non-null  float64
 6   DAYS_BIRTH                    307511 non-null  int64  
 7   DAYS_EMPLOYED                 307511 non-null  int64  
 8   DAYS_REGISTRATION             307511 non-null  float64
 9   DAYS_ID_PUBLISH               307511 non-null  int64  
 10  OWN_CAR_AGE                   104582 non-null  float64
 11  FLAG_EMP_PHONE                307511 non-null  int64  
 12  FLAG_WORK_PHONE               307511 non-nul

In [11]:
# Passo 1: Preparação dos Dados
# Passo 2: Treinamento da Árvore de Decisão
# Passo 3: Avaliação do Modelo

X = df_arvore_2.drop(columns=['TARGET'])  # Todas as variáveis menos o target
y = df_arvore_2['TARGET']  # O target

# Dividir os dados em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar o modelo de árvore de decisão
decision_tree = DecisionTreeClassifier(random_state=42)

# Treinar o modelo com os dados de treino
decision_tree.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred = decision_tree.predict(X_test)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Exibir os resultados
print(f'Acurácia: {accuracy:.4f}')
print(f'Precisão: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')
print('\nMatriz de Confusão:')
print(confusion_matrix(y_test, y_pred))
print('\nRelatório de Classificação:')
print(classification_report(y_test, y_pred))

Acurácia: 0.8423
Precisão: 0.1017
Recall: 0.1224
F1-Score: 0.1111

Matriz de Confusão:
[[51200  5354]
 [ 4343   606]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.92      0.91      0.91     56554
           1       0.10      0.12      0.11      4949

    accuracy                           0.84     61503
   macro avg       0.51      0.51      0.51     61503
weighted avg       0.86      0.84      0.85     61503



In [12]:
# Treinamento da árvore com class_weight balanceado
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Parâmetros a serem ajustados
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced']
}

# Modelo de árvore
decision_tree = DecisionTreeClassifier(random_state=42)

# Grid Search com validação cruzada
grid_search = GridSearchCV(estimator=decision_tree, param_grid=param_grid, cv=5, scoring='f1', verbose=1, n_jobs=-1)

# Treinamento
grid_search.fit(X_train, y_train)

# Avaliação
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print(f"Melhores parâmetros: {grid_search.best_params_}")
print(f'Acurácia: {accuracy_score(y_test, y_pred):.4f}')
print(f'Precisão: {precision_score(y_test, y_pred):.4f}')
print(f'Recall: {recall_score(y_test, y_pred):.4f}')
print(f'F1-Score: {f1_score(y_test, y_pred):.4f}')

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Melhores parâmetros: {'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}
Acurácia: 0.5934
Precisão: 0.1125
Recall: 0.5886
F1-Score: 0.1890


Ao comparar os dois cenários, podemos avaliar o desempenho do modelo de detecção de fraudes:

1. **Árvore original**:
   - Acurácia: 0.8423
   - Precisão: 0.1017
   - Recall: 0.1224
   - F1-Score: 0.1111

   A árvore original tem uma boa acurácia (84%), mas a precisão e o recall para a classe fraudulenta (classe 1) são muito baixos. Isso indica que o modelo está identificando poucas fraudes e, quando identifica, muitos dos casos são falsos positivos. O F1-Score de 0.1111 reflete o equilíbrio pobre entre precisão e recall.

2. **Árvore com class_weight='balanced'**:
   - Acurácia: 0.5934
   - Precisão: 0.1125
   - Recall: 0.5886
   - F1-Score: 0.1890

   Ao ajustar o modelo para `class_weight='balanced'`, houve uma redução significativa na acurácia geral (59%), o que era esperado, já que o modelo está sendo ajustado para equilibrar as classes. O recall para a classe 1 (fraudes) melhorou muito, subindo para 58.9%, o que significa que o modelo está identificando mais fraudes. No entanto, a precisão continua baixa (11.25%), indicando que muitos falsos positivos ainda estão sendo gerados. O F1-Score de 0.1890 é melhor que o anterior, mostrando que o modelo está conseguindo melhorar o equilíbrio entre precisão e recall.

### Conclusão:
Embora a acurácia tenha diminuído na segunda abordagem, o aumento no recall é crucial para detecção de fraudes, pois identificar o maior número possível de fraudes (mesmo com alguns falsos positivos) é muitas vezes mais importante do que manter uma acurácia alta. O modelo com `class_weight='balanced'` parece ser mais adequado para detecção de fraudes, especialmente se o objetivo for minimizar fraudes não detectadas. Para melhorar ainda mais, pode ser interessante explorar outras técnicas de ajuste de hiperparâmetros ou algoritmos, como Random Forest ou XGBoost.

## Árvore 3 - Tentativa melhoria (a melhor até entao rs)

In [17]:
# Carregar os dados
df3 = pd.read_csv('application_data.csv')

In [18]:
# criação da coluna NUM_DOCUMENTS_PROVIDED

document_columns = ['FLAG_DOCUMENT_' + str(i) for i in range(2, 22)]
df3['NUM_DOCUMENTS_PROVIDED'] = df3[document_columns].sum(axis=1)

# exclusao das colunas FLAG_DOCUMENT_X
column = ['FLAG_DOCUMENT_2',
       'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5',
       'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8',
       'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11',
       'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14',
       'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17',
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21']
df3 = df3.drop(column, axis=1)

# exclusao de colunas com mais de 19% de dados nulos
columns_to_drop = ['OWN_CAR_AGE', 'EXT_SOURCE_1', 'EXT_SOURCE_3', 'EXT_SOURCE_3','APARTMENTS_AVG', 'BASEMENTAREA_AVG', 
                   'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG',
                   'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG',
                   'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE',
                   'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE',
                   'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE',
                   'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE',
                   'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI',
                   'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI',
                   'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI',
                   'NONLIVINGAREA_MEDI', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'TOTALAREA_MODE',
                   'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']
df3 = df3.drop(columns=columns_to_drop)



In [21]:
# Tratamento dos valores nulos 
numerical_columns_low_missing = ['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'CNT_FAM_MEMBERS', 'EXT_SOURCE_2',
                                  'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
                                  'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE']
df3[numerical_columns_low_missing] = df3[numerical_columns_low_missing].fillna(df3[numerical_columns_low_missing].median())

df3['OCCUPATION_TYPE'] = df3['OCCUPATION_TYPE'].fillna('unknown')
df3['NAME_TYPE_SUITE'] = df3['NAME_TYPE_SUITE'].fillna(df3['NAME_TYPE_SUITE'].mode()[0])

rows_to_drop = df3[(df3['TARGET'] == 0) & df3[['AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']].isnull().any(axis=1)].index
df3 = df3.drop(rows_to_drop)

df3['AMT_REQ_CREDIT_BUREAU_YEAR'] = pd.to_numeric(df3['AMT_REQ_CREDIT_BUREAU_YEAR'], errors='coerce')

bin_edges = [0, 5, 10, 20]  # Adjust the bin edges as needed
bin_labels = ['Low from 0 to 5', 'Medium from 6 to 10', 'High +11']

df3['AMT_REQ_CREDIT_BUREAU_YEAR_Binned'] = pd.cut(df3['AMT_REQ_CREDIT_BUREAU_YEAR'], bins=bin_edges, labels=bin_labels, right=False)

unknown_category = 'unknown'
df3['AMT_REQ_CREDIT_BUREAU_YEAR_Binned'] = df3['AMT_REQ_CREDIT_BUREAU_YEAR_Binned'].cat.add_categories([unknown_category])
df3['AMT_REQ_CREDIT_BUREAU_YEAR_Binned'] = df3['AMT_REQ_CREDIT_BUREAU_YEAR_Binned'].fillna(unknown_category)
bin_counts = df3['AMT_REQ_CREDIT_BUREAU_YEAR_Binned'].value_counts().sort_index()


In [23]:
# Convert the column to numeric, coercing errors to NaN
df3['AMT_REQ_CREDIT_BUREAU_QRT'] = pd.to_numeric(df3['AMT_REQ_CREDIT_BUREAU_QRT'], errors='coerce')

# Define bin edges and labels
bin_edges = [0, 5, float('inf')]  # Adjust the bin edges as needed
bin_labels = ['Low from 0 to 5', 'High above 5']

# Create a new column with bins
df3['AMT_REQ_CREDIT_BUREAU_QRT_Binned'] = pd.cut(df3['AMT_REQ_CREDIT_BUREAU_QRT'], bins=bin_edges, labels=bin_labels, right=False)

unknown_category = 'unknown'
df3['AMT_REQ_CREDIT_BUREAU_QRT_Binned'] = df3['AMT_REQ_CREDIT_BUREAU_QRT_Binned'].cat.add_categories([unknown_category])
df3['AMT_REQ_CREDIT_BUREAU_QRT_Binned'] = df3['AMT_REQ_CREDIT_BUREAU_QRT_Binned'].fillna(unknown_category)

# Display the counts in each bin
bin_counts = df3['AMT_REQ_CREDIT_BUREAU_QRT_Binned'].value_counts().sort_index()

# Convert the column to numeric, coercing errors to NaN
df3['AMT_REQ_CREDIT_BUREAU_MON'] = pd.to_numeric(df3['AMT_REQ_CREDIT_BUREAU_MON'], errors='coerce')

# Define bin edges and labels
bin_edges = [0, 5, 10, float('inf')]  # Adjust the bin edges as needed
bin_labels = ['Low from 0 to 5', 'Medium from 6 to 10', 'High +10']

# Create a new column with bins
df3['AMT_REQ_CREDIT_BUREAU_MON_Binned'] = pd.cut(df3['AMT_REQ_CREDIT_BUREAU_MON'], bins=bin_edges, labels=bin_labels, right=False)

unknown_category = 'unknown'
df3['AMT_REQ_CREDIT_BUREAU_MON_Binned'] = df3['AMT_REQ_CREDIT_BUREAU_MON_Binned'].cat.add_categories([unknown_category])
df3['AMT_REQ_CREDIT_BUREAU_MON_Binned'] = df3['AMT_REQ_CREDIT_BUREAU_MON_Binned'].fillna(unknown_category)

# Display the counts in each bin
bin_counts = df3['AMT_REQ_CREDIT_BUREAU_MON_Binned'].value_counts().sort_index()

columns_to_process = ['AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_HOUR']
df3[columns_to_process] = df3[columns_to_process].apply(pd.to_numeric, errors='coerce')


In [24]:
# Define bin edges and labels
bin_edges = [0, 2, float('inf')]  # Adjust the bin edges as needed
bin_labels = ['Low from 0 to 2', 'High above 2']

# Iterate over columns
for column in columns_to_process:
    # Create a new column with bins
    bin_column_name = f'{column}_Binned'
    df3[bin_column_name] = pd.cut(df3[column], bins=bin_edges, labels=bin_labels, right=False)

    # Add an 'unknown' category
    unknown_category = 'unknown'
    df3[bin_column_name] = df3[bin_column_name].cat.add_categories([unknown_category])

    # Fill missing values with 'unknown'
    df3[bin_column_name] = df3[bin_column_name].fillna(unknown_category)

    # Display the counts in each bin
    bin_counts = df3[bin_column_name].value_counts().sort_index()
#    print(f"Counts for {bin_column_name}:")
#    print(bin_counts)

column = ['AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON',
         'AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR']
df3 = df3.drop(column, axis=1)

categorical_columns = ['FLAG_MOBIL', 'FLAG_EMP_PHONE', 'SK_ID_CURR','FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'NUM_DOCUMENTS_PROVIDED',
                       'REGION_RATING_CLIENT','REGION_RATING_CLIENT_W_CITY','REG_CITY_NOT_LIVE_CITY','REG_CITY_NOT_WORK_CITY',
                      'REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION','LIVE_REGION_NOT_WORK_REGION','LIVE_CITY_NOT_WORK_CITY']

df3[categorical_columns] = df3[categorical_columns].astype('category')

df3['SK_ID_CURR'] = df3['SK_ID_CURR'].astype('object')

columns_to_remove_sign = ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH','DAYS_LAST_PHONE_CHANGE']
df3[columns_to_remove_sign] = df3[columns_to_remove_sign].abs()

In [25]:
# Feature engineering
df3['AGE'] = df3['DAYS_BIRTH'] // 365
df3['YEARS_EMPLOYED'] = df3['DAYS_EMPLOYED'] // 365
df3['YEARS_REGISTRATION'] = df3['DAYS_REGISTRATION'] // 365
df3['YEARS_ID_PUBLISH'] = df3['DAYS_ID_PUBLISH'] // 365
df3['YEARS_LAST_PHONE_CHANGE'] =df3['DAYS_LAST_PHONE_CHANGE'] /365

# Define bin edges and labels for AGE_Binned
age_bin_edges = [18, 30, 40, 50, 60, 70, 80]
age_bin_labels = ['18-30', '31-40', '41-50', '51-60', '61-70', '+71']

# Define bin edges and labels for YEARS_EMPLOYED_Binned and YEARS_REGISTRATION_Binned
employment_bin_edges = [0, 5, 12, 20, float('inf')]
employment_bin_labels = ['0-5', '6-12', '13-20', '+25']

# Apply binning to the newly created features
df3['AGE_Binned'] = pd.cut(df3['AGE'], bins=age_bin_edges, labels=age_bin_labels, right=False)
df3['YEARS_EMPLOYED_Binned'] = pd.cut(df3['YEARS_EMPLOYED'], bins=employment_bin_edges, labels=employment_bin_labels, right=False)
df3['YEARS_REGISTRATION_Binned'] = pd.cut(df3['YEARS_REGISTRATION'], bins=employment_bin_edges, labels=employment_bin_labels, right=False)
df3['YEARS_ID_PUBLISH_Binned'] = pd.cut(df3['YEARS_ID_PUBLISH'], bins=age_bin_edges, labels=age_bin_labels, right=False)

# Drop original columns
df3 = df3.drop(columns_to_remove_sign, axis=1)

bin_edges = [0, 5, 10, float('inf')]
bin_labels = ['0-5', '6-10', '11+']

df3['OBS_30_CNT_SOCIAL_CIRCLE_Binned'] = pd.cut(df3['OBS_30_CNT_SOCIAL_CIRCLE'], bins=bin_edges, labels=bin_labels, right=False)
df3['OBS_60_CNT_SOCIAL_CIRCLE_Binned'] = pd.cut(df3['OBS_60_CNT_SOCIAL_CIRCLE'], bins=bin_edges, labels=bin_labels, right=False)
df3['DEF_30_CNT_SOCIAL_CIRCLE_Binned'] = pd.cut(df3['DEF_30_CNT_SOCIAL_CIRCLE'], bins=bin_edges, labels=bin_labels, right=False)
df3['DEF_60_CNT_SOCIAL_CIRCLE_Binned'] = pd.cut(df3['DEF_60_CNT_SOCIAL_CIRCLE'], bins=bin_edges, labels=bin_labels, right=False)

for col in ['OBS_30_CNT_SOCIAL_CIRCLE_Binned', 'OBS_60_CNT_SOCIAL_CIRCLE_Binned', 
            'DEF_30_CNT_SOCIAL_CIRCLE_Binned', 'DEF_60_CNT_SOCIAL_CIRCLE_Binned']:
    bin_counts = df3[col].value_counts().sort_index()
    print(f"{col}:\n{bin_counts}\n")

df3 = df3.drop(['OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 
              'DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE'], axis=1)


OBS_30_CNT_SOCIAL_CIRCLE_Binned:
OBS_30_CNT_SOCIAL_CIRCLE_Binned
0-5     244096
6-10     22518
11+       3670
Name: count, dtype: int64

OBS_60_CNT_SOCIAL_CIRCLE_Binned:
OBS_60_CNT_SOCIAL_CIRCLE_Binned
0-5     244544
6-10     22184
11+       3556
Name: count, dtype: int64

DEF_30_CNT_SOCIAL_CIRCLE_Binned:
DEF_30_CNT_SOCIAL_CIRCLE_Binned
0-5     270222
6-10        61
11+          1
Name: count, dtype: int64

DEF_60_CNT_SOCIAL_CIRCLE_Binned:
DEF_60_CNT_SOCIAL_CIRCLE_Binned
0-5     270262
6-10        21
11+          1
Name: count, dtype: int64



In [26]:
columns_to_drop = ['YEARS_ID_PUBLISH_Binned', 'YEARS_ID_PUBLISH', 'FLAG_MOBIL',
                   'FLAG_EMP_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_EMAIL','REG_REGION_NOT_WORK_REGION',
                   'REG_REGION_NOT_LIVE_REGION','REG_CITY_NOT_WORK_CITY','REG_CITY_NOT_LIVE_CITY']
df3 = df3.drop(columns_to_drop,axis=1)
df3 = df3[df3['CODE_GENDER'] != 'XNA']

In [27]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 270280 entries, 0 to 307510
Data columns (total 47 columns):
 #   Column                             Non-Null Count   Dtype   
---  ------                             --------------   -----   
 0   SK_ID_CURR                         270280 non-null  object  
 1   TARGET                             270280 non-null  int64   
 2   NAME_CONTRACT_TYPE                 270280 non-null  object  
 3   CODE_GENDER                        270280 non-null  object  
 4   FLAG_OWN_CAR                       270280 non-null  object  
 5   FLAG_OWN_REALTY                    270280 non-null  object  
 6   CNT_CHILDREN                       270280 non-null  int64   
 7   AMT_INCOME_TOTAL                   270280 non-null  float64 
 8   AMT_CREDIT                         270280 non-null  float64 
 9   AMT_ANNUITY                        270280 non-null  float64 
 10  AMT_GOODS_PRICE                    270280 non-null  float64 
 11  NAME_TYPE_SUITE                

In [28]:
# Selecionar colunas numéricas (int64 e float64)
col_numericas = df3.select_dtypes(include=['int64', 'float64']).columns
numericas = []

for coluna in col_numericas:
    numericas.append(coluna)

df_arvore_3 = df3[numericas]
print('base nova: ', df_arvore_3.info())

<class 'pandas.core.frame.DataFrame'>
Index: 270280 entries, 0 to 307510
Data columns (total 14 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   TARGET                      270280 non-null  int64  
 1   CNT_CHILDREN                270280 non-null  int64  
 2   AMT_INCOME_TOTAL            270280 non-null  float64
 3   AMT_CREDIT                  270280 non-null  float64
 4   AMT_ANNUITY                 270280 non-null  float64
 5   AMT_GOODS_PRICE             270280 non-null  float64
 6   REGION_POPULATION_RELATIVE  270280 non-null  float64
 7   CNT_FAM_MEMBERS             270280 non-null  float64
 8   HOUR_APPR_PROCESS_START     270280 non-null  int64  
 9   EXT_SOURCE_2                270280 non-null  float64
 10  AGE                         270280 non-null  int64  
 11  YEARS_EMPLOYED              270280 non-null  int64  
 12  YEARS_REGISTRATION          270280 non-null  float64
 13  YEARS_LAST_PHONE_CH

In [29]:
# Definir as features e o target
X = df_arvore_3.drop(columns=['TARGET'])  # Todas as variáveis menos o target
y = df_arvore_3['TARGET']  # O target

# Dividir os dados em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar o modelo de árvore de decisão
decision_tree = DecisionTreeClassifier(random_state=42)

# Treinar o modelo com os dados de treino
decision_tree.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred = decision_tree.predict(X_test)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Exibir os resultados
print(f'Acurácia: {accuracy:.4f}')
print(f'Precisão: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')
print('\nMatriz de Confusão:')
print(confusion_matrix(y_test, y_pred))
print('\nRelatório de Classificação:')
print(classification_report(y_test, y_pred))

Acurácia: 0.8296
Precisão: 0.1364
Recall: 0.1649
F1-Score: 0.1493

Matriz de Confusão:
[[44039  5116]
 [ 4093   808]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.91      0.90      0.91     49155
           1       0.14      0.16      0.15      4901

    accuracy                           0.83     54056
   macro avg       0.53      0.53      0.53     54056
weighted avg       0.84      0.83      0.84     54056



In [30]:
# Treinamento da árvore com class_weight balanceado
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Parâmetros a serem ajustados
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced']
}

# Modelo de árvore
decision_tree = DecisionTreeClassifier(random_state=42)

# Grid Search com validação cruzada
grid_search = GridSearchCV(estimator=decision_tree, param_grid=param_grid, cv=5, scoring='f1', verbose=1, n_jobs=-1)

# Treinamento
grid_search.fit(X_train, y_train)

# Avaliação
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print(f"Melhores parâmetros: {grid_search.best_params_}")
print(f'Acurácia: {accuracy_score(y_test, y_pred):.4f}')
print(f'Precisão: {precision_score(y_test, y_pred):.4f}')
print(f'Recall: {recall_score(y_test, y_pred):.4f}')
print(f'F1-Score: {f1_score(y_test, y_pred):.4f}')

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Melhores parâmetros: {'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}
Acurácia: 0.6397
Precisão: 0.1496
Recall: 0.6346
F1-Score: 0.2421


Ao analisar os resultados dos dois cenários:

### 1. **Árvore original**:
- **Acurácia**: 0.8296
- **Precisão**: 0.1364
- **Recall**: 0.1649
- **F1-Score**: 0.1493
- **Matriz de Confusão**:
  - Verdadeiros Negativos (classe 0): 44,039
  - Falsos Positivos: 5,116
  - Falsos Negativos: 4,093
  - Verdadeiros Positivos (classe 1): 808

Neste cenário, a acurácia é boa (82.96%), mas a **precisão** e o **recall** para a classe de fraudes (classe 1) ainda são baixos, indicando que o modelo está identificando poucas fraudes corretamente. O F1-Score de 0.1493 mostra que o modelo tem dificuldade em equilibrar a precisão e o recall para a detecção de fraudes.

### 2. **Árvore com `class_weight='balanced'`**:
- **Acurácia**: 0.6397
- **Precisão**: 0.1496
- **Recall**: 0.6346
- **F1-Score**: 0.2421

Após aplicar o ajuste de `class_weight='balanced'`, a acurácia diminuiu significativamente para 63.97%. No entanto, houve uma grande melhora no **recall** (63.46%), o que significa que o modelo está conseguindo identificar a maior parte das fraudes. A **precisão** ainda é relativamente baixa (14.96%), o que indica a presença de falsos positivos. No entanto, o F1-Score melhorou para 0.2421, o que mostra que o modelo está conseguindo um melhor equilíbrio entre precisão e recall.

### **Conclusão**:
O modelo com `class_weight='balanced'` é mais adequado para a detecção de fraudes. Mesmo com a queda na acurácia, o aumento significativo no recall é importante, pois ele reflete a capacidade do modelo de identificar fraudes. Embora ainda haja um número considerável de falsos positivos, o aumento no recall é geralmente mais desejável em um cenário de detecção de fraudes, onde é preferível detectar mais fraudes, mesmo que com algum custo de precisão. O F1-Score mais alto (0.2421) também sugere que o desempenho geral na identificação de fraudes melhorou com a abordagem balanceada.